In [1]:
import torch
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import cv2
from cifar10_models import mobilenetv2
from torchvision import transforms, datasets
from torch.utils.data import Subset, DataLoader
import matplotlib.pyplot as plt

In [2]:
device = (torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu'))
print(f"Training on device {device}.")

Training on device cuda.


In [3]:

Net = mobilenetv2.mobilenet_v2(pretrained=True)

In [5]:
for module in Net.modules():
    if isinstance(module, nn.BatchNorm2d):
        module.eval()

In [ ]:
#for param in Net.parameters():
#    param.requires_grad = False


In [5]:
preprocess = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4915, 0.4823, 0.4468), (0.2470, 0.2435, 0.2616))
])

In [6]:
torch.cuda.memory_allocated()/1024**2, torch.cuda.memory_cached()/1024**2

(0.0, 0.0)

In [7]:
class RegressionModel(nn.Module):
    def __init__(self, pretrainedModel):
        super().__init__()
        self.features = pretrainedModel.features
        
        self.Flatten = nn.Flatten()
        self.FC1 = nn.Linear(20480, 256)
        self.FC2 = nn.Linear(256, 128)
        self.FC3 = nn.Linear(128, 64)
        self.FC4 = nn.Linear(64, 32)


        self.Dropout2D = nn.Dropout2d(p=0.2)
        self.Dropout = nn.Dropout(p=0.2)


        self.outLayer1 = nn.Linear(32, 4)
        self.outLayer2 = nn.Linear(32, 4)
        self.outLayer3 = nn.Linear(32, 4)
        self.outLayer4 = nn.Linear(32, 4)
    
    def forward(self, x):
        x = self.Dropout2D(self.features(x))
        x = self.Flatten(x) # flatten out the last conv layer

        x = self.Dropout(torch.selu(self.FC1(x)))# use dropout with p=0.2
        x = self.Dropout(torch.selu(self.FC2(x)))
        x = self.Dropout(torch.selu(self.FC3(x)))
        x = self.Dropout(torch.selu(self.FC4(x)))

        out1 = self.outLayer1(x)
        out2 = self.outLayer2(x)
        out3 = self.outLayer3(x)
        out4 = self.outLayer4(x)

        return out1, out2, out3, out4


In [8]:
dataset = datasets.CIFAR10('../data-unversioned/p1ch7/', train=True, transform=preprocess, download=False)
imgs = os.listdir('data/training')
imgs.sort()
indices = [int(name[0:5]) for name in imgs]
my_subset = Subset(dataset, indices) #create subset based on indices
train_loader = DataLoader(my_subset, batch_size=1, shuffle=False) #shuffle false so we can use correct stuff

In [9]:
labels = torch.from_numpy(np.load('data/labels.npy')).float()
labels = labels.to(device=device)

In [10]:
model = RegressionModel(Net).to(device=device)


In [11]:
img = cv2.imread('data/attacks/00000_input_img.png')
imgT = torch.tensor(img)
imgT = imgT.view(3, 32 , 32).to(device=device)
imgT = torch.unsqueeze(imgT, 0).float()
out1, out2, out3, out4 = model(imgT)
model.train()
out1[0].shape, out2.shape

(torch.Size([4]), torch.Size([1, 4]))

In [12]:
model.training

True

In [13]:
loss = nn.MSELoss()

In [14]:
optimizer = optim.AdamW(model.parameters(), lr=1e-4)

In [15]:
def training_loop(n_epochs, optimizer, model, loss_fn, loader, labels):    
    for epoch in range(1, n_epochs+1):
        loss_train = 0.0
        idx = 0
        for img_batch, _ in loader:
            
            img_batch = img_batch.to(device=device)
            out1, out2, out3, out4 = model(img_batch)
            loss1 = loss_fn(out1[0], labels[idx][0])
            loss2 = loss_fn(out2[0], labels[idx][1])
            loss3 = loss_fn(out3[0], labels[idx][2])
            loss4 = loss_fn(out4[0], labels[idx][3])

            loss_total = (loss1 + loss2 + loss3 + loss4) #accumulate loss

            optimizer.zero_grad()

            loss_total.backward()
            optimizer.step()

            
            
            if idx % 400 == 0:
                print(f'step {idx} is the current iteration and loss is: {loss_total}')

            idx += 1
        
        print(f'At epoch: {epoch}, the training loss is {loss_total}')
            

In [16]:
training_loop(n_epochs=2, optimizer=optimizer, model=model, loss_fn=loss, loader=train_loader, labels=labels)

step 0 is the current iteration and loss is: 598.6073608398438
step 400 is the current iteration and loss is: 159.17478942871094
step 800 is the current iteration and loss is: 249.94680786132812
step 1200 is the current iteration and loss is: 54.236473083496094
step 1600 is the current iteration and loss is: 133.2995147705078
step 2000 is the current iteration and loss is: 223.1544952392578
step 2400 is the current iteration and loss is: 82.50814819335938
step 2800 is the current iteration and loss is: 48.13749694824219
step 3200 is the current iteration and loss is: 69.76011657714844
step 3600 is the current iteration and loss is: 112.3565444946289
step 4000 is the current iteration and loss is: 119.34650421142578
step 4400 is the current iteration and loss is: 72.93921661376953
At epoch: 1, the training loss is 102.05825805664062
step 0 is the current iteration and loss is: 139.89503479003906
step 400 is the current iteration and loss is: 59.45940399169922
step 800 is the current ite

In [ ]:
asd = my_subset[5][0].to(device=device)

In [ ]:
del model
torch.cuda.empty_cache()
del labels
torch.cuda.empty_cache()

In [17]:
model.eval()
print("hello")

hello


In [18]:
model.training

False

In [23]:
asd2 = my_subset[16][0].to(device=device)
asd3 = my_subset[67][0].to(device=device)

In [24]:
model(torch.unsqueeze(asd2, 0)), model(torch.unsqueeze(asd3, 0))

((tensor([[13.0863, 14.7098,  2.8455,  2.4467]], device='cuda:0',
         grad_fn=<AddmmBackward>),
  tensor([[13.3207, 14.1652,  4.0221,  3.7565]], device='cuda:0',
         grad_fn=<AddmmBackward>),
  tensor([[13.8847, 14.1954,  3.6099,  3.4445]], device='cuda:0',
         grad_fn=<AddmmBackward>),
  tensor([[14.3875, 13.7179,  4.3074,  4.3675]], device='cuda:0',
         grad_fn=<AddmmBackward>)),
 (tensor([[13.5272, 15.1986,  2.8975,  2.5543]], device='cuda:0',
         grad_fn=<AddmmBackward>),
  tensor([[13.6988, 14.6159,  4.1145,  3.8615]], device='cuda:0',
         grad_fn=<AddmmBackward>),
  tensor([[14.3594, 14.6630,  3.6989,  3.5497]], device='cuda:0',
         grad_fn=<AddmmBackward>),
  tensor([[14.7970, 14.1685,  4.4152,  4.4691]], device='cuda:0',
         grad_fn=<AddmmBackward>)))

In [21]:
labels[2]

tensor([[21.6715, 19.8500,  5.0000,  3.8596],
        [16.2501, 26.6525,  5.0000,  5.0000],
        [12.6542, 22.0432,  5.0000,  3.9232],
        [ 5.7360,  9.2632,  5.0000,  4.9370]], device='cuda:0')